In [47]:
from IPython.core.display import display, HTML
from scipy.spatial.transform import Rotation
display(HTML("<style>.container { width:100% !important; }</style>"))
import h5py
from matplotlib import pyplot as plt
import torch
import numpy as np
from pathlib import Path
import glob
import os
import h5py
import numpy as np
import prody
import prody as pr
from potsim2 import PotGrid
import sklearn
from sklearn.metrics import matthews_corrcoef as mcc 
from sklearn.metrics import f1_score
import os
import pandas as pd


import matplotlib
#matplotlib.use('TkAgg')

%matplotlib inline

In [55]:
def yieldData(predbase, testData):

    for fname in glob.glob(f'{testData}/*/*_pocket.pdb'):
        pocketfname = Path(fname)
        name = pocketfname.parent.name
        predfname = f'{predbase}/{name}/pocket_pred.pdb'
        pdbfname = f'{testData}/{name}/{name}_selected.pdb'

        
        pocket = prody.parsePDB(pocketfname)
        structure = prody.parsePDB(pdbfname)

        try:
            pred = prody.parsePDB(predfname)
            yield name, (structure, pred, pocket)
            
        except OSError:
            if os.path.exists(predfname+'.empty'):
                print(f'No pockets predicted for {name}')
                yield name, (structure, None, pocket)
            else:
                print(f'Prediction for name {name} was not found')
        

        
def calcMetric(pred, pocket, structure, scoref):
    if pred is None:
        prednums = set()
    else:
        prednums = set(t.getResnum() for t in pred.iterResidues())
    truenums = set(t.getResnum() for t in pocket.iterResidues())
    allnums = set(t.getResnum() for t in structure.iterResidues())

    tp = 0
    tn = 0
    fp = 0
    fn = 0

    y_true = []
    y_pred = []

    for num in allnums:
        if num not in prednums and num not in truenums:
            tn += 1
            y_pred.append(-1)
            y_true.append(-1)
        elif num not in prednums and num in truenums:
            fn += 1
            y_pred.append(-1)
            y_true.append(1)
        elif num in prednums and num not in truenums:
            fp += 1
            y_pred.append(1)
            y_true.append(-1)
        elif num in prednums and num in truenums:
            tp += 1
            y_pred.append(1)
            y_true.append(1)
        else:
            raise Exception

    return scoref(y_true, y_pred)

def scoresFromGen(gen):
    names = []
    mcc_scores = []
    f_scores = []
    for name, (structure, pred, pocket) in gen:
        yield name, (calcMetric(pred, pocket, structure, f1_score), calcMetric(pred, pocket, structure, mcc))

In [57]:
testData = f'/home/lorenzo/3dunet-cavity/procData_210821'

predname = '210817_features_cpu'
predbase = f'/home/lorenzo/3dunet-cavity/runs/{predname}/predictions'

predbaseKala = '/home/lorenzo/kalasanty/predictions_210821'

In [72]:
resOurs = list(scoresFromGen(yieldData(predbase, testData)))
resTheirs = list(scoresFromGen(yieldData(predbaseKala, testData)))

/home/lorenzo/miniconda3/envs/apbs/lib/python3.8/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


No pockets predicted for 2jdu
No pockets predicted for 4pf5


/home/lorenzo/miniconda3/envs/apbs/lib/python3.8/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/lorenzo/miniconda3/envs/apbs/lib/python3.8/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


No pockets predicted for 3t1a


/home/lorenzo/miniconda3/envs/apbs/lib/python3.8/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


...

/home/lorenzo/miniconda3/envs/apbs/lib/python3.8/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


No pockets predicted for 2epn


/home/lorenzo/miniconda3/envs/apbs/lib/python3.8/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


No pockets predicted for 4rd6


/home/lorenzo/miniconda3/envs/apbs/lib/python3.8/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [74]:
def mergeDict(dx, dy):
    keys = set(dx.keys()) | set(dy.keys())
    return {key:(dx.get(key),dy.get(key)) for key in keys}

dictOurs = {x[0]: x[1] for x in resOurs}
dictTheirs = {x[0]: x[1] for x in resTheirs}
d2 = mergeDict(dictOurs, dictTheirs)

d2

{'3v2n': ((0.5, 0.5), (0.4347826086956522, 0.4330127018922193)),
 '4q19': ((0.5846153846153845, 0.5546156826565473),
  (0.6984126984126985, 0.6878760347820342)),
 '1kyv': ((0.7213114754098361, 0.6981293686592875),
  (0.8076923076923077, 0.7811777612289629)),
 '2q89': ((0.6666666666666666, 0.6832312780114154),
  (0.7692307692307693, 0.7654407656447454)),
 '5fnu': ((0.49350649350649356, 0.5033394714036898),
  (0.5, 0.49050443909780655)),
 '5laq': ((0.39999999999999997, 0.3586235957978007),
  (0.6129032258064516, 0.6175548671479134)),
 '1pgp': ((0.8148148148148149, 0.8139468677571629), (0.0, 0.0)),
 '4jwk': ((0.375, 0.40154633339543605),
  (0.4827586206896552, 0.4440089017395818)),
 '3rlr': ((0.5454545454545455, 0.5252470469857199),
  (0.6666666666666665, 0.6431229781880295)),
 '3s0e': ((0.6071428571428572, 0.5660428772005275),
  (0.7555555555555556, 0.7232711335067522)),
 '1g30': ((0.823529411764706, 0.8032626942802905),
  (0.0, -0.06920518032979132)),
 '1n4k': ((0.5116279069767442, 0.54

In [77]:
# f1_score ours
np.mean([x[0][0] for k,x in d2.items()]), np.std([x[0][0] for k,x in d2.items()])

(0.49820489992400047, 0.21281642874906717)

In [78]:
# f1_score theirs
np.mean([x[1][0] for k,x in d2.items()]), np.std([x[1][0] for k,x in d2.items()])

(0.4429008739237578, 0.3284758926389769)

In [81]:
# mcc ours
np.mean([x[0][1] for k,x in d2.items()]), np.std([x[0][1] for k,x in d2.items()])

(0.48482094871703, 0.22937120875770634)

In [82]:
# mcc theirs
np.mean([x[1][1] for k,x in d2.items()]), np.std([x[1][1] for k,x in d2.items()])

(0.41869395956563593, 0.34744801749667464)